# Prilagajanje modelov Open AI

Ta zvezek temelji na trenutnih navodilih, ki jih najdete v dokumentaciji [Prilagajanje](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) Open AI.

Prilagajanje izboljša zmogljivost osnovnih modelov za vašo aplikacijo z dodatnim učenjem na dodatnih podatkih in kontekstu, ki je pomemben za določen primer uporabe ali scenarij. Upoštevajte, da tehnike oblikovanja pozivov, kot sta _few shot learning_ in _retrieval augmented generation_, omogočajo izboljšanje privzetega poziva z ustreznimi podatki za izboljšanje kakovosti. Vendar pa so ti pristopi omejeni z največjo velikostjo okna za število tokenov ciljanega osnovnega modela.

S prilagajanjem dejansko ponovno učimo model z zahtevanimi podatki (kar nam omogoča uporabo veliko več primerov, kot jih lahko sprejme največje okno za število tokenov) - in uvajamo _prilagojeno_ različico modela, ki ne potrebuje več primerov med izvajanjem. To ne le izboljša učinkovitost oblikovanja poziva (imamo večjo prilagodljivost pri uporabi okna za število tokenov za druge stvari), ampak potencialno tudi zniža stroške (z zmanjšanjem števila tokenov, ki jih moramo poslati modelu med izvajanjem).

Prilagajanje ima 4 korake:
1. Pripravite učne podatke in jih naložite.
1. Zaženite učno nalogo, da dobite prilagojen model.
1. Ocenite prilagojeni model in ponavljajte za izboljšanje kakovosti.
1. Uvedite prilagojeni model za izvajanje, ko ste zadovoljni.

Upoštevajte, da ne podpirajo vsi osnovni modeli prilagajanja - [preverite dokumentacijo OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) za najnovejše informacije. Prav tako lahko prilagodite že prej prilagojen model. V tem vodiču bomo kot ciljni osnovni model za prilagajanje uporabili `gpt-35-turbo`. 

---


### Korak 1.1: Pripravite svoj nabor podatkov

Zgradimo klepetalnega robota, ki vam pomaga razumeti periodni sistem elementov tako, da odgovarja na vprašanja o elementu z limeriko. V _tem_ preprostem vodiču bomo ustvarili le nabor podatkov za usposabljanje modela z nekaj primeri odgovorov, ki prikazujejo pričakovani format podatkov. V resničnem primeru uporabe bi morali ustvariti nabor podatkov z veliko več primeri. Prav tako boste morda lahko uporabili odprt nabor podatkov (za vaše področje uporabe), če obstaja, in ga preoblikovali za uporabo pri fino nastavitvi.

Ker se osredotočamo na `gpt-35-turbo` in iščemo enkratni odgovor (zaključek klepeta), lahko ustvarimo primere z uporabo [tega predlaganega formata](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), ki odraža zahteve OpenAI za zaključke klepeta. Če pričakujete večkratno pogovorno vsebino, bi uporabili [format primerov za večkratne pogovore](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), ki vključuje parameter `weight` za označevanje, katere sporočila naj se uporabljajo (ali ne) v postopku fino nastavitve.

Za naš vodič bomo uporabili preprostejši format enkratnega odgovora. Podatki so v [jsonl formatu](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) z enim zapisom na vrstico, vsak predstavljen kot objekt v JSON obliki. Spodnji odlomek prikazuje 2 zapisa kot primer - glejte [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) za celoten vzorec (10 primerov), ki ga bomo uporabili za naš vodič fino nastavitve. **Opomba:** Vsak zapis _mora_ biti definiran v eni sami vrstici (ne razdeljen čez več vrstic, kot je običajno v formatirani JSON datoteki)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

V resničnem primeru uporabe boste potrebovali veliko večji nabor primerov za dobre rezultate - kompromis bo med kakovostjo odgovorov in časom/stroški fino nastavitve. Uporabljamo majhen nabor, da lahko hitro zaključimo fino nastavitev in prikažemo postopek. Glejte [ta primer iz OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) za bolj zapleten vodič fino nastavitve.


---

### Korak 1.2 Naložite svoj nabor podatkov

Naložite podatke z uporabo Files API [kot je opisano tukaj](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Upoštevajte, da morate za zagon te kode najprej opraviti naslednje korake:
 - Namestiti Python paket `openai` (poskrbite, da uporabljate različico >=0.28.0 za najnovejše funkcije)
 - Nastaviti okoljsko spremenljivko `OPENAI_API_KEY` na vaš OpenAI API ključ
Za več informacij si oglejte [vodnik za nastavitev](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), ki je na voljo za tečaj.

Zdaj zaženite kodo, da ustvarite datoteko za nalaganje iz vaše lokalne JSONL datoteke.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Korak 2.1: Ustvarite nalogo za fino nastavljanje z SDK-jem


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Korak 2.2: Preverite stanje naloge

Tukaj je nekaj stvari, ki jih lahko naredite z API-jem `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Prikaži zadnjih n nalog za fino nastavljanje
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Pridobi podrobnosti o določeni nalogi za fino nastavljanje
- `client.fine_tuning.jobs.cancel(<job_id>)` - Prekliči nalogo za fino nastavljanje
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Prikaži do n dogodkov iz naloge
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvi korak procesa je _preverjanje veljavnosti datoteke za učenje_, da se zagotovi, da so podatki v pravilni obliki.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Korak 2.3: Spremljajte dogodke za nadzor napredka


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Korak 2.4: Ogled statusa v nadzorni plošči OpenAI


Status si lahko ogledate tudi tako, da obiščete spletno stran OpenAI in raziščete razdelek _Fine-tuning_ na platformi. Ta vam bo prikazal status trenutnega opravila in vam omogočil tudi sledenje zgodovine preteklih izvedb opravil. Na tem posnetku zaslona lahko vidite, da je prejšnja izvedba spodletela, druga pa je bila uspešna. Za kontekst, to se je zgodilo, ko je prva izvedba uporabila JSON datoteko z nepravilno oblikovanimi zapisi – ko je bilo to popravljeno, je druga izvedba uspešno zaključila in model naredila na voljo za uporabo.

![Fine-tuning job status](../../../../../translated_images/sl/fine-tuned-model-status.563271727bf7bfba.png)


Statusna sporočila in metrike si lahko ogledate tudi tako, da se v vizualni nadzorni plošči pomaknete še nižje, kot je prikazano:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/sl/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/sl/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Korak 3.1: Pridobite ID in preizkusite fino nastavljeni model v kodi


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Korak 3.2: Naložite in preizkusite fino nastavljeni model v Playgroundu

Zdaj lahko fino nastavljeni model preizkusite na dva načina. Najprej lahko obiščete Playground in v spustnem meniju Models izberete vaš novo fino nastavljeni model med ponujenimi možnostmi. Druga možnost je uporaba možnosti "Playground", prikazane v panelu Fine-tuning (glejte posnetek zaslona zgoraj), ki za hitro oceno zažene naslednji _primerjalni_ pogled, ki prikazuje osnovno in fino nastavljeno različico modela ena ob drugi.

![Fine-tuning job status](../../../../../translated_images/sl/fine-tuned-playground-compare.56e06f0ad8922016.png)

Preprosto vnesite sistemski kontekst, uporabljen v vaših učnih podatkih, in podajte vaše testno vprašanje. Opazili boste, da se obe strani posodobita z enakim kontekstom in vprašanjem. Zaženite primerjavo in videli boste razliko v izhodih med njima. _Opazite, kako fino nastavljeni model oblikuje odgovor v formatu, ki ste ga navedli v svojih primerih, medtem ko osnovni model preprosto sledi sistemskemu pozivu_.

![Fine-tuning job status](../../../../../translated_images/sl/fine-tuned-playground-launch.5a26495c983c6350.png)

Opazili boste, da primerjava prav tako prikazuje število tokenov za vsak model in čas, potreben za inferenco. **Ta specifični primer je poenostavljen in namenjen prikazu procesa, ne pa dejanskemu odražanju resničnega nabora podatkov ali scenarija**. Morda boste opazili, da oba vzorca prikazujeta enako število tokenov (sistemski kontekst in uporabniški poziv sta enaka), pri čemer fino nastavljeni model potrebuje več časa za inferenco (prilagojeni model).

V resničnih scenarijih ne boste uporabljali takšnega preprostega primera, ampak boste fino nastavljali na resničnih podatkih (npr. katalog izdelkov za podporo strankam), kjer bo kakovost odgovora veliko bolj očitna. V _tem_ kontekstu bo za dosego enakovredne kakovosti odgovora z osnovnim modelom potrebna bolj prilagojena inženiring pozivov, kar bo povečalo uporabo tokenov in potencialno tudi čas obdelave za inferenco. _Za preizkus si oglejte primere fino nastavljanja v OpenAI Cookbook, da začnete._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Za morebitna nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda, ne odgovarjamo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
